In [ ]:
# HAllo???
# 123

### Notebook purpose
- Connect to Overpass API
- Extract coordinates of all hiking trails within Switzerland
- Convert data into a pandas dataframe object
- Create a table in SQL database (hosted on Microsoft Azure)
- Store coordinates in SQL DB

overpy ist eine Python-Bibliothek, die es ermöglicht, Daten von der Overpass API (eine Schnittstelle für OpenStreetMap-Daten) abzufragen und zu verarbeiten. Die Overpass API ermöglicht es, Wanderwege aus dem OpenStreetMap-Projekt (OSM) abzurufen.

Die Abrage sucht nach Wanderrouten in einem Giebiet welche mit spezifischen Signalisationen ausgeschildert sind. 

Falls das der Name nicht vorhanden ist, aber die Bezeichnungen von und bis existieren, wird der Name zusammengesetzt.



sqlalchemy dient dazu, SQL-Datenbanken mit Python-Code zu verbinden und erleichtert das Arbeiten mit relationalen Datenbanken. sqlalchemy bietet einen direkten Zugriff auf SQL-Datenbanken, was Flexibilität ermöglicht.

In [1]:
# Import required libraries
import pandas as pd 
import overpy
import json
import os
from sqlalchemy import create_engine
import pymssql

In [2]:
# Initialisiere die Overpass API mit einer benutzerdefinierten URL
api = overpy.Overpass(url="http://overpass.osm.ch/api/interpreter")

# Overpass Query für Wanderwege innerhalb der Schweiz. Mit Center erhalten wir die Koordinaten in der Mitte eines Wnaderweges
query = """
[out:json];
relation
["route"="hiking"]
/*["name"]*/
["name"!~"fixme", i]
["network"="lwn"]
["osmc:symbol"~"yellow::yellow_diamond|red:white:red_bar|yellow:white:yellow_diamond|blue:white:blue_bar"]
/*(id: 1432463)*/
(45.8899, 6.0872, 47.8085, 10.4921);
out center tags;
"""

# Führe die Anfrage aus
result = api.query(query)

# Liste zum Speichern der extrahierten Informationen
list = []

# Iteration über alle relations
for relation in result.relations:
    
    # Extrahiere die relevanten Daten

    org_name = relation.tags.get('name')
    fix_name = ""
    org_to = relation.tags.get('to')
    org_from = relation.tags.get('from')
    
    # Center ist ein Tupel mit langitute und longitude, wir möchten nur einen wert
    lat = getattr(relation, 'center_lat')
    lon = getattr(relation, 'center_lon')
    
    # Wenn der orginal Name nicht vorhanden ist setze ihn aus from und to zusammen
    if not org_name and org_from and org_to :
        fix_name = f"{org_from} - {org_to}"
    else:
        fix_name = org_name

    # Erstelle ein Dictionary um die Attribute als Tupel zu speichern
    if fix_name and lat > 0 and lon > 0: 
        dict = {
        'id': relation.id,
        'name': fix_name,
        'lat': lat,
        'lon': lon
        }

        # Jedes Tupel wird nun in der Liste als neue Zeile gespeichert
        list.append(dict)

# Wenn alle Daten verarbeitet wurden, erstelle das DataFrame
df_wanderwege = pd.DataFrame(list)


In [3]:
# Add time and datestamp of API call to dataframe
df_wanderwege["timestamp_apicall"] = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")

# Change column order and print dataframe
col_order = ['timestamp_apicall', 'id', 'name', 'lat', 'lon']
df_wanderwege = df_wanderwege[col_order]

print(df_wanderwege.info())
print("-----------------------------------------")
print(df_wanderwege.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15196 entries, 0 to 15195
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   timestamp_apicall  15196 non-null  object
 1   id                 15196 non-null  int64 
 2   name               15196 non-null  object
 3   lat                15196 non-null  object
 4   lon                15196 non-null  object
dtypes: int64(1), object(4)
memory usage: 593.7+ KB
None
-----------------------------------------
     timestamp_apicall      id                                          name  \
0  2024-09-20 10:53:34   22614  Nationalpark Wanderroute 15 (Munt la Schera)   
1  2024-09-20 10:53:34  103607                                 Wanderwege SG   
2  2024-09-20 10:53:34  112830                Uetliberg - Uetliberg Uto Kulm   
3  2024-09-20 10:53:34  112831                           Folenweid - Baldern   
4  2024-09-20 10:53:34  112833                          Felseneg

In [4]:
# Get current working directory
current_dir = os.getcwd()
print(current_dir)

# c:\Users\etien\OneDrive\02_Progression\CAS_DataEngineering_ZHAW\03_Leistungsnachweis\Wanderwege\notebooks

c:\Users\etien\OneDrive\02_Progression\CAS_DataEngineering_ZHAW\03_Leistungsnachweis\Wanderwege\notebooks


In [5]:
# Load database access configuration from config/db_config.json
with open('../config/db_config.json', 'r') as f:
    db_config = json.load(f)

# Access db credentials
server = db_config['server']
database = db_config['database']
db_user = db_config['db_user']
db_password = db_config['db_password']

In [8]:
# Connect to SQL Database
conn = pymssql.connect(server, db_user, db_password, database)

In [10]:
# Create connection string for SQLAlchemy
connection_string = f"mssql+pymssql://{db_user}:{db_password}@{server}/{database}"
engine = create_engine(connection_string)

In [ ]:
# Write DataFrame to SQL Database
df_wanderwege.to_sql('wanderwege', con=engine, if_exists='replace', index=False)

In [ ]:
df_test = pd.read_sql_table(table_name='wanderwege', con=engine)
print(df_test.head(5))

In [49]:
# Explicitly close the engine's connection pool (not best practice -> add "with engine.connect() as connection:" instead)
engine.dispose()

In [ ]:
# Maybe use this function for multiple loads
""" 

def load_data_to_sql(df, table_name, server, database, username, password):
    # Build the connection string
    connection_string = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
    
    # Create the engine
    params = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')
    
    # Use inspector to check if the table exists
    inspector = inspect(engine)
    
    # Check if the table exists in the database
    if table_name in inspector.get_table_names():
        print(f"Table '{table_name}' exists. Appending data...")
        # Append the DataFrame to the existing table
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
    else:
        print(f"Table '{table_name}' does not exist. Creating table and inserting data...")
        # Create the table and insert the data
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    
    print("Data has been loaded successfully.")
    
"""

In [ ]:
### Example - Load DataFrame into MySQL Database

import pandas as pd
from sqlalchemy import create_engine

# Verbindung zur MySQL-Datenbank herstellen (ersetze Platzhalter mit deinen Daten)
user = 'root'        # MySQL-Benutzername
password = ''  # MySQL-Passwort
host = '127.0.0.1'   # Server (bei lokalen Installationen ist das oft 'localhost')
database = 'mydatabase'  # Name der Datenbank, in die du schreiben möchtest

# Erstelle die Verbindungs-Engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# DataFrame in die MySQL-Datenbank-Tabelle schreiben (wenn die Tabelle nicht existiert, wird sie erstellt)
df.to_sql(name='Wanderwege', con=engine, if_exists='replace', index=False)

print("DataFrame erfolgreich in die MySQL-Datenbank geladen!")